## Занятие 7: Докинг низкомолекулярных лигандов в структуру белка

### Лев Мазаев, мАДБМ18

### Загрузка модулей

In [ ]:
import numpy as np

# Loading Open Drug Discovery Toolkit
import oddt
import oddt.docking
import oddt.interactions

# Structure and Viewing
from Bio.PDB import *
from ipywidgets import VBox
import nglview as nv
import IPython.display as ipd

In [ ]:
def viewer(*args):
    # функция для трехмерного отображения моделей
    models = []
    for pdb in args:
        v = nv.show_structure_file(pdb)
        v._remote_call('setSize', target='Widget', args=['800px', '600px'])
        v.representations = [
            {"type": "cartoon", "params": {"sele": "protein", "color": "residueindex"}},
            {"type": "ball+stick", "params": {"sele": "ligand"}}
        ]
        models.append(v)
    return(VBox(models))

### Подготовка белка

Визуализируем структуру белка с лигандом:

In [ ]:
view1 = viewer('lys_pig_ligand.B99990002.pdb')
view1

Загрузим структуру и посмотрим остатки:

In [ ]:
parser = PDBParser()
pdb = parser.get_structure('lys_pig_ligand', 'lys_pig_ligand.B99990002.pdb')

for res in pdb.get_residues():
    if res.id[1] > 140:
        print(res)

147-149 остатки в структуре - лиганд. Создадим отдельные объекты белка и лиганда, визуализируем их:

In [ ]:
class NonHetSelect(Select):
    
    def accept_residue(self, residue):
        return 1 if residue.id[0] == ' ' else 0
    
class HetSelect(Select):
    
    def accept_residue(self, residue):
        return 0 if residue.id[0] == ' ' else 1
    
io = PDBIO()
io.set_structure(pdb)
io.save('protein.pdb', NonHetSelect())
io.save('ligand.pdb', HetSelect())

view2 = viewer('protein.pdb', 'ligand.pdb')
view2

Найдем геометрический центр лиганда:

In [ ]:
ligand = parser.get_structure('ligand', 'ligand.pdb')
atom_coords = []

for atom in ligand.get_atoms():
    v = atom.get_vector()
    atom_coords.append(v.get_array())
    
geom_center = np.mean(atom_coords, axis=0)
print(geom_center)

### Подготовка белка для докинга

In [ ]:
protein = next(oddt.toolkit.readfile('pdb', 'protein.pdb'))
protein.OBMol.AddPolarHydrogens()
protein.OBMol.AutomaticPartialCharge()

In [ ]:
protein

In [ ]:
print(f'Is it protein? {protein.protein}')
print(f'Molecular weight: {round(protein.molwt, 3)}')
protein.protein = True  # исправим

### Лиганды для докинга

Отобразим исходный лиганд:

In [ ]:
ligand = next(oddt.toolkit.readfile('pdb', 'ligand.pdb'))
ligand.OBMol.AddPolarHydrogens()
ligand.OBMol.AutomaticPartialCharge()
ipd.SVG(ligand.write('svg', opt={'d': None, 'P': 500}))

По заданию не совсем понятно, нужно провести докинг только модифицированного NAG или всего лиганда целиком (NAG-NAG-NDG) с модифицированными NAG. Решил сделать второе. Исходная SMILES-формула:

In [ ]:
ligand.smiles

In [ ]:
smiles = f'[C@@H]1([C@@H]([C@H]([C@@H]([C@@H](CO)O1)O)O)NC(=O){"O"})O[C@H]1[C@@H]([C@H]([C@@H](O[C@@H]1CO)O[C@H]1[C@@H]([C@H]([C@H](O[C@@H]1CO)O)NC(=O){"[NH3+]"})O)NC(=O){""})O'

In [ ]:
smiles

In [ ]:
m = oddt.toolkit.readstring('smi', smiles)
if not m.OBMol.Has3D():
    m.make3D(forcefield='mmff94', steps=500)
    m.removeh()
    m.OBMol.AddPolarHydrogens()
    
ipd.SVG(m.write('svg', opt={'d': None, 'P': 500}))

In [ ]:
m = oddt.toolkit.readstring('smi', 'c1cccc(O)c1')

if not m.OBMol.Has3D():
    m.make3D(forcefield='mmff94', steps=500)
    m.removeh()
    m.OBMol.AddPolarHydrogens()
    
ipd.SVG(m.write('svg', opt={'d': None, 'P': 500}))

### Докинг

In [ ]:
dock_obj = oddt.docking.AutodockVina.autodock_vina(
    protein=protein, size=(10, 10, 10), center=geom_center,
    executable='/home/leo/Software/vina/vina', autocleanup=True, num_modes=5
)

In [ ]:
print(dock_obj.tmp_dir)

In [ ]:
print(' '.join(dock_obj.params))

Здесь перечислены параметры докинга:

- center(x, y, z) - центр окна докинга, в данном случае совпадает с геометрическим центром исходного лиганда
- size(x, y, z) - размеры окна докинга, куб с соответствующими сторонами
- exhaustiveness - отвечает за время поиска
- num_modes - 
- energy_range 

In [ ]:
protein.protein

In [ ]:
res = dock_obj.dock([m], protein)

In [ ]:
for i, r in enumerate(res):
    print(f'{i}, {r.formula}, {r.data["vina_affinity"]}, {r.data["vina_rmsd_ub"]}, {r.residues[0].name}')

In [ ]:
for i, r in enumerate(res):
    hbs = oddt.interactions.hbonds(protein, r)
    stack = oddt.interactions.pi_stacking(protein, r)
    phob = oddt.interactions.hydrophobic_contacts(protein, r)

In [ ]:
model_list = []
for i, r in enumerate(res):
    r.write(filename=f'{i}.pdb', format='pdb')
    model_list.append(f'{i}.pdb')

In [ ]:
viewer(*model_list)